In [35]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


## Get Notes from Music Files

In [4]:
midi = converter.parse("midi_songs/dontbeafraid.mid")

In [5]:
midi

<music21.stream.Score 0x7f9f16ee8358>

In [9]:
midi.show('midi')

In [8]:
midi.show('text')

{0.0} <music21.stream.Part 0x7f9f16a9acf8>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.tempo.MetronomeMark Quarter=163.0>
    {0.0} <music21.tempo.MetronomeMark Quarter=163.0>
    {0.0} <music21.key.Key of d minor>
    {0.0} <music21.meter.TimeSignature 5/4>
    {0.0} <music21.stream.Voice 0x7f9f16c9ecf8>
        {0.0} <music21.chord.Chord A3 D4>
        {1.0} <music21.chord.Chord A3 D4>
        {1.5} <music21.chord.Chord A3 D4>
        {2.5} <music21.chord.Chord A3 D4>
        {3.0} <music21.chord.Chord A3 D4>
        {4.0} <music21.chord.Chord A3 D4>
        {5.0} <music21.chord.Chord A3 C4 D4>
        {6.0} <music21.chord.Chord A3 C4 D4>
        {6.5} <music21.chord.Chord A3 C4 D4>
        {7.5} <music21.chord.Chord A3 C4 D4>
        {8.0} <music21.chord.Chord A3 C4 D4>
        {9.0} <music21.chord.Chord A3 C4 D4>
        {10.0} <music21.chord.Chord B3 D4>
        {11.0} <music21.chord.Chord B3 D4>
        {11.5} <music21.chord.Chord B3 D4>
        {12.5} <music2

In [15]:
for i in midi.parts:
    print(i[0], i)

Piano <music21.stream.Part 0x7f9f16a9acf8>


In [16]:
for i in midi.flat:
    print(i)

Piano
<music21.tempo.MetronomeMark Quarter=163.0>
<music21.tempo.MetronomeMark Quarter=163.0>
d minor
<music21.meter.TimeSignature 5/4>
<music21.chord.Chord A3 D4>
<music21.note.Note D>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 C4 D4>
<music21.note.Note D>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord B3 D4>
<music21.note.Note D>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B-3 D4>
<music21.note.Note D>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord A3 D4>
<music21.note.Note D>
<music21.chord.Chord A3 D4>
<m

In [18]:
midi.flat.notes.show('text')

{0.0} <music21.chord.Chord A3 D4>
{0.0} <music21.note.Note D>
{1.0} <music21.chord.Chord A3 D4>
{1.5} <music21.chord.Chord A3 D4>
{2.5} <music21.chord.Chord A3 D4>
{3.0} <music21.chord.Chord A3 D4>
{4.0} <music21.chord.Chord A3 D4>
{5.0} <music21.chord.Chord A3 C4 D4>
{5.0} <music21.note.Note D>
{6.0} <music21.chord.Chord A3 C4 D4>
{6.5} <music21.chord.Chord A3 C4 D4>
{7.5} <music21.chord.Chord A3 C4 D4>
{8.0} <music21.chord.Chord A3 C4 D4>
{9.0} <music21.chord.Chord A3 C4 D4>
{10.0} <music21.chord.Chord B3 D4>
{10.0} <music21.note.Note D>
{11.0} <music21.chord.Chord B3 D4>
{11.5} <music21.chord.Chord B3 D4>
{12.5} <music21.chord.Chord B3 D4>
{13.0} <music21.chord.Chord B3 D4>
{14.0} <music21.chord.Chord B3 D4>
{15.0} <music21.chord.Chord B-3 D4>
{15.0} <music21.note.Note D>
{16.0} <music21.chord.Chord B-3 D4>
{16.5} <music21.chord.Chord B-3 D4>
{17.5} <music21.chord.Chord B-3 D4>
{18.0} <music21.chord.Chord B-3 D4>
{19.0} <music21.chord.Chord B-3 D4>
{20.0} <music21.chord.Chord A3 D4>

In [17]:
len(midi.flat.notes)

618

In [19]:
notes_to_parse = midi.flat.notes

In [20]:
for i in notes_to_parse:
    print(i, i.offset)

<music21.chord.Chord A3 D4> 0.0
<music21.note.Note D> 0.0
<music21.chord.Chord A3 D4> 1.0
<music21.chord.Chord A3 D4> 1.5
<music21.chord.Chord A3 D4> 2.5
<music21.chord.Chord A3 D4> 3.0
<music21.chord.Chord A3 D4> 4.0
<music21.chord.Chord A3 C4 D4> 5.0
<music21.note.Note D> 5.0
<music21.chord.Chord A3 C4 D4> 6.0
<music21.chord.Chord A3 C4 D4> 6.5
<music21.chord.Chord A3 C4 D4> 7.5
<music21.chord.Chord A3 C4 D4> 8.0
<music21.chord.Chord A3 C4 D4> 9.0
<music21.chord.Chord B3 D4> 10.0
<music21.note.Note D> 10.0
<music21.chord.Chord B3 D4> 11.0
<music21.chord.Chord B3 D4> 11.5
<music21.chord.Chord B3 D4> 12.5
<music21.chord.Chord B3 D4> 13.0
<music21.chord.Chord B3 D4> 14.0
<music21.chord.Chord B-3 D4> 15.0
<music21.note.Note D> 15.0
<music21.chord.Chord B-3 D4> 16.0
<music21.chord.Chord B-3 D4> 16.5
<music21.chord.Chord B-3 D4> 17.5
<music21.chord.Chord B-3 D4> 18.0
<music21.chord.Chord B-3 D4> 19.0
<music21.chord.Chord A3 D4> 20.0
<music21.note.Note D> 20.0
<music21.chord.Chord A3 D4> 21

In [21]:
notes_to_parse[0]

<music21.chord.Chord A3 D4>

In [26]:
notes_demo= []
for element in notes_to_parse:
    # If the element is a Note, then store it's pitch
    if isinstance(element, note.Note): 
        notes_demo.append(str(element.pitch))

    # If the element is a Chord, split each note of the chord and join them with dot
    elif isinstance(element, chord.Chord): 
        notes_demo.append('+'.join(str(n) for n in element.normalOrder))

In [27]:
isinstance(notes_to_parse[1], note.Note)

True

In [28]:
notes_to_parse[0].normalOrder

[9, 2]

In [29]:
notes_to_parse[0]

<music21.chord.Chord A3 D4>

In [30]:
notes_demo

['9+2',
 'D3',
 '9+2',
 '9+2',
 '9+2',
 '9+2',
 '9+2',
 '9+0+2',
 'D3',
 '9+0+2',
 '9+0+2',
 '9+0+2',
 '9+0+2',
 '9+0+2',
 '11+2',
 'D3',
 '11+2',
 '11+2',
 '11+2',
 '11+2',
 '11+2',
 '10+2',
 'D3',
 '10+2',
 '10+2',
 '10+2',
 '10+2',
 '10+2',
 '9+2',
 'D3',
 '9+2',
 '9+2',
 '9+2',
 '9+2',
 '9+2',
 '9+0+2',
 'D3',
 '9+0+2',
 '9+0+2',
 '9+0+2',
 '9+0+2',
 '9+0+2',
 '7+11+2',
 'D3',
 '7+11+2',
 '7+11+2',
 '7+11+2',
 '7+11+2',
 '7+11+2',
 '10+2+5',
 'D3',
 '10+2+5',
 '10+2+5',
 '10+2+5',
 '10+2+5',
 '10+2+5',
 'A4',
 'E4',
 'A2',
 'B4',
 'A2',
 '9+0',
 '9+2',
 'A4',
 'E4',
 'A2',
 'E4',
 'E4',
 '3+9',
 'E-4',
 'E-4',
 'D4',
 'A2',
 'D4',
 '9+0',
 '5+9',
 'A2',
 'F5',
 'B4',
 'A2',
 'D5',
 '9+0+4',
 '9+11+2',
 '2+5',
 'A2',
 'D5',
 'A2',
 'C5',
 '9+11',
 '9',
 '11+2',
 'G2',
 '0+4',
 '11+2',
 '9+0',
 '7+11',
 '9+0',
 '7+11',
 '2+5',
 '5+9',
 'G2',
 'D3',
 '4+7',
 'E3',
 '2+5',
 'F3',
 '11+2',
 'G3',
 '9+0+4',
 '9+2',
 '9+0+4',
 '2+6+9',
 '9+0+4',
 '2+6+9',
 '4+7+9',
 '6+9',
 'A4',
 'E4',
 

In [31]:
len(notes_demo)

618

In [33]:
len(set(notes_demo))

55

In [32]:
np.unique(notes_demo, return_counts=True)

(array(['0+4', '0+5', '10+2', '10+2+5', '11+2', '2+5', '2+6', '2+6+9',
        '3+9', '4', '4+5', '4+7', '4+7+9', '4+9', '5', '5+9', '5+9+0',
        '6+9', '7', '7+11', '7+11+2', '7+9', '9', '9+0', '9+0+2', '9+0+4',
        '9+11', '9+11+2', '9+2', 'A2', 'A3', 'A4', 'A5', 'B-4', 'B3', 'B4',
        'B5', 'C4', 'C5', 'C6', 'D3', 'D4', 'D5', 'D6', 'E-4', 'E3', 'E4',
        'E5', 'F#5', 'F2', 'F3', 'F5', 'G2', 'G3', 'G5'], dtype='<U6'),
 array([  2,   2,   6,   8,  12,  11,   6,   8,   5,   2,   2,   2,   4,
          8,   2,  11,   2,   6,   2,   6,   6,  10,   5,  20,  12,  17,
          7,   5,  29, 134,   6,  12,   8,   2,   4,  16,   6,   2,   7,
          2,  44,  18,  12,   2,  10,  10,  24,   2,   4,  32,  10,   5,
          8,  14,   6]))

In [36]:
notes = []


for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file) #convert the file into stream.Score Object

    print("Parsing %s" % file)

    notes_to_parse = None  
    
    # unroll / flat the elements (notes/chords) into the list, because elements are sometimes in list of list form.
    notes_to_parse = midi.flat.notes   
    
    
    # At this point, we have notes_to_parse, which is a Iterator for all the Notes/Chords
    
    for element in notes_to_parse:
        
        # If the element is a Note, then store it's pitch
        if isinstance(element, note.Note): 
            notes.append(str(element.pitch))
            
        # If the element is a Chord, split each note of the chord and join them with +
        elif isinstance(element, chord.Chord): 
            notes.append('+'.join(str(n) for n in element.normalOrder))

Parsing midi_songs/Rachel_Piano_tempofix.mid
Parsing midi_songs/ff11_awakening_piano.mid
Parsing midi_songs/Eternal_Harvest.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing midi_songs/FF3_Battle_(Piano).mid
Parsing midi_songs/ViviinAlexandria.mid
Parsing midi_songs/waltz_de_choco.mid
Parsing midi_songs/great_war.mid
Parsing midi_songs/Oppressed.mid
Parsing midi_songs/path_of_repentance.mid
Parsing midi_songs/goldsaucer.mid
Parsing midi_songs/ultimafro.mid
Parsing midi_songs/pkelite4.mid
Parsing midi_songs/decisive.mid
Parsing midi_songs/gerudo.mid
Parsing midi_songs/FF6epitaph_piano.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/sandy.mid
Parsing midi_songs/tpirtsd-piano.mid
Parsing midi_songs/Ff7-Cinco.mid
Parsing midi_songs/Rydia_pc.mid
Parsing midi_songs/In_Zanarkand.mid
Parsing midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing midi_songs/figaro.mid
Parsing midi_songs/Kingdom_Hearts_Dearly_Beloved.mid
Parsing midi_song

In [37]:
with open('data/notes_easy', 'wb') as filepath:
    pickle.dump(notes, filepath)

In [38]:
with open('data/notes_easy', 'rb') as filepath:
    notes = pickle.load(filepath)

In [39]:
n_vocab = len(set(notes))

In [40]:
print("Total notes- ",len(notes))
print("Unique Notes- ",n_vocab)

Total notes-  60498
Unique Notes-  359


In [41]:
# notes is a list of all the notes in each music file - 60000
print(notes[:100])

['G4', 'G5', 'B4', 'E5', 'G4', 'G5', 'F#4', 'F#5', 'E4', 'E5', 'D4', 'D5', 'F#4', 'F#5', 'E4', 'E5', 'C5', 'A3', 'C4', 'E4', 'C5', 'C4', 'B4', 'G3', 'A4', 'B3', 'G4', 'D4', 'B4', 'B3', 'A4', 'F3', 'A3', 'C4', 'E4', 'G4', 'F5', 'F5', 'A4', 'D5', 'F5', 'F5', 'E5', 'E5', 'D5', 'D5', 'C5', 'C5', 'E5', 'E5', 'D5', 'D5', 'B-4', 'B-4', 'C5', 'C5', 'D5', 'D4', 'B-4', 'D4', 'A4', 'D4', 'G4', 'D4', 'G3', 'D4', 'G4', 'D4', 'B-4', 'F4', 'D4', 'C5', 'E4', 'C4', 'D5', 'D3', 'G3', 'A3', 'G4', 'A4', 'G4', 'D4', 'A3', 'D3', 'G5', 'G4', 'A4', 'B-4', 'G4', 'D5', 'D4', 'A4', 'G5', 'B-4', 'G4', 'A5', 'C4', 'G4', 'A4', 'F4']


## Prepare Data for LSTM

In [42]:
#  How many elements LSTM input should consider
sequence_length = 100

In [43]:
# ALl the unique elements in a sorted manner
pitchnames = sorted(set(item for item in notes))

In [44]:
# Mapping between note to int value
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

{'0': 0, '0+1': 1, '0+1+5': 2, '0+1+6': 3, '0+2': 4, '0+2+3+7': 5, '0+2+4+7': 6, '0+2+5': 7, '0+2+6': 8, '0+2+7': 9, '0+3': 10, '0+3+5': 11, '0+3+5+8': 12, '0+3+6': 13, '0+3+6+8': 14, '0+3+6+9': 15, '0+3+7': 16, '0+4': 17, '0+4+5': 18, '0+4+6': 19, '0+4+7': 20, '0+5': 21, '0+5+6': 22, '0+6': 23, '1': 24, '1+2': 25, '1+2+4+6+8+10': 26, '1+2+6': 27, '1+2+6+8': 28, '1+3': 29, '1+3+5': 30, '1+3+5+8': 31, '1+3+6': 32, '1+3+7': 33, '1+3+8': 34, '1+4': 35, '1+4+6': 36, '1+4+6+9': 37, '1+4+7': 38, '1+4+7+10': 39, '1+4+7+9': 40, '1+4+8': 41, '1+5': 42, '1+5+8': 43, '1+5+9': 44, '1+6': 45, '1+7': 46, '10': 47, '10+0': 48, '10+0+2+5': 49, '10+0+3': 50, '10+0+4': 51, '10+0+5': 52, '10+1': 53, '10+1+3': 54, '10+1+3+5+6': 55, '10+1+3+6': 56, '10+1+4': 57, '10+1+4+6': 58, '10+1+5': 59, '10+11': 60, '10+11+3': 61, '10+11+3+5': 62, '10+2': 63, '10+2+3': 64, '10+2+4': 65, '10+2+5': 66, '10+3': 67, '11': 68, '11+0': 69, '11+0+4': 70, '11+0+4+6': 71, '11+0+4+7': 72, '11+0+5': 73, '11+1': 74, '11+1+4': 75,

In [45]:
network_input = []
network_output = []

In [46]:
# Make the data for LSTM Network
# Each Node will contain 100 input units
# Output will be the next unit in notes list

for i in range(0, len(notes) - sequence_length, 1):
    
    sequence_in = notes[i:i + sequence_length] # contains 100 values 
    sequence_out = notes[i + sequence_length] # containes next values for these 100's
    
    # Since NN works with numeric data only, append the int values for inputs and outputs.
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

In [47]:
# No. of examples for our network

n_patterns = len(network_input)
print(n_patterns)

60398


In [48]:
# ight now network_input is lits, but reshape it into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(60398, 100, 1)


In [49]:
normalised_network_input = network_input/float(n_vocab)

In [50]:
# network_output are the classes, encode one_hot_vector
network_output = np_utils.to_categorical(network_output)

In [51]:
print(network_output.shape)
print(normalised_network_input.shape)

(60398, 359)
(60398, 100, 1)


## Create Model

In [52]:
""" create the structure of the neural network """
model = Sequential()
model.add(LSTM(512,
            input_shape=(network_input.shape[1], network_input.shape[2]),
            return_sequences=True ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

W0811 14:04:56.939704 140322256295744 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 14:04:56.983942 140322256295744 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 14:04:56.987332 140322256295744 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 14:04:57.487859 140322256295744 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is depre

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

In [56]:
""" train the neural network """

filepath = "model/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min')

callbacks_list = [checkpoint]

model_his = model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

In [55]:
model = load_model("model/weights-improvement-60-1.1406-bigger.hdf5")

## Predictions

In [57]:
#  This is done beacuse we need network_input as list, and we had converted it into ndarray to feed into network
#  so, to get back the list.

sequence_length = 100
network_input = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])

In [58]:
""" Generate notes from the neural network based on a sequence of notes """

# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)

# Mapping from int to note
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

# initial pattern
pattern = network_input[start]
prediction_output = []

# generate 200 notes
for note_index in range(200):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [60]:
#  Our newly generated song (mix of chords and notes)
print(prediction_output)

['C5', '2+5', '2+5', 'F5', '2+5', '2+5', '2+5', 'F5', '2+5', 'C5', '0+5', '2+5', 'F5', '2+5', '2+5', '2+5', '2+5', 'G5', '2+5', '2+5', 'G5', '2+5', '2+5', 'G5', '2+5', '2+5', '2+5', '2+5', '2+5', '2+5', 'F5', '2+5', 'C5', 'F5', '2+5', '2+5', 'G5', '2+5', '2+5', 'F5', '2+5', 'C5', 'F5', '0+5', '2+5', '2+5', '2+5', '0+4', '2+5', 'F5', '2+5', '2+5', 'F5', '2+5', '2+5', 'G5', '2+5', '2+5', '0+5', '2+5', '2+5', '0+5', '0+5', '2+5', '0+5', '2+5', '2+5', '2+5', 'G5', '2+5', '2+5', 'G5', '2+5', '2+5', 'F5', '2+5', '2+5', 'G5', '2+5', '2+5', 'F5', '2+5', '2+5', '2+5', '2+5', '0+5', '0+5', 'F5', '0+5', '0+5', 'F5', 'C5', 'B4', '2+5', '2+5', '2+5', '2+5', '2+5', '0+5', '0+5', 'F5', '0+5', '0+5', 'B4', '0+5', '0+5', '0+5', '2+5', '0+5', '0+5', '0+5', '0+5', 'F5', '0+5', '0+5', '0+5', '0+5', '2+5', '0+5', '0+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', 'C5', '0+5', '2+5', '2+5', 'B-2', '0+5', '2+5', 'E3', '0+5', '0+5', 'F#4', '0+5', '0+5', 'E5', '0+5', '0+5', '0+5', '0+

## Create MIDI file

In [61]:
""" convert the output from the prediction to notes and create a midi file from the notes """

offset = 0 # Time
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    
    # if pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+') # split all the notes from a chord
        notes = []
        for current_note in notes_in_chord: 
            new_note = note.Note(int(current_note)) # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano() 
            notes.append(new_note) # list of Notes()
            
        new_chord = chord.Chord(notes)  # Create the Chord() from the list of notes.
        new_chord.offset = offset # set offset to the element
        output_notes.append(new_chord) 
        
    # if pattern is a note
    else:
        new_note = note.Note(pattern) #  create Note object
        new_note.offset = offset # set offset  (basically timestamp)
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note) 

    # increase offset each iteration so that notes do not stack (time stamp)
    offset += 0.5

In [62]:
output_notes[:50]

[<music21.note.Note C>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note F>,
 <music21.chord.Chord D F>,
 <music21.note.Note C>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord D F>,
 <music21.note.Note F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note F>,
 <music21.chord.Chord D F>,
 <music21.note.Note C>,
 <music21.note.Note F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.ch

In [63]:
#  Create a Steam Object from our generated Notes, and write to the file. 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output_songs/test_output.mid')

'output_songs/test_output.mid'

## Let's Play our music file

In [64]:
midi_stream.show('midi')